In [327]:
import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

# Part 1: Getting the Neighborhoods from Wikipedia

## Scrape the table from the webpage

In [232]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
postal_codes=pd.read_html(url)

In [233]:
pc=postal_codes[0]
pc

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Clean the table

First let's see how many postal codes are missing boroughs

In [234]:
na=pc[pc['Borough']=='Not assigned']
na

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


Now let's convert 'Not assigned' to numpy's NaN, so the rows can be dropped, and then drop those rows.

In [235]:
pc.Borough.replace({'Not assigned':np.nan}, inplace=True)

In [236]:
pc.head()

,Postal Code,Borough,Neighbourhood
0,M1A,NaN,Not assigned
1,M2A,NaN,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [237]:
pc.dropna(axis=0, inplace=True)

## Let's the view the cleaned dataframe

In [238]:
pc

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


... and its size:

In [239]:
pc.shape

(103, 3)

# Part 2: Latitude and Longitude

I tried the suggestd with no luck. I also tried the geopy.geocoder Nominatim from the New York clustering lab, but that gave poor results for Canadian postal codes, so I decided to use the csv file provided.

In [240]:
coord=pd.read_csv('/Users/matthewyoung/Documents/Education - Personal/IBM Data Science Coursera Course/Data Downloads/Geospatial_Coordinates.csv')

In [241]:
coord=coord.sort_values('Postal Code')
print('Coordinates:')
coord.head()

Coordinates:


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Sort the neighborhoods dataframe by postal code and reset the index so that it will match up with the lat/long dataframe.

In [242]:
pc=pc.sort_values('Postal Code')
print('Neighborhoods:')
pc.reset_index(inplace=True)
pc.head()

Neighborhoods:


,index,Postal Code,Borough,Neighbourhood
0,9,M1B,Scarborough,"Malvern, Rouge"
1,18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,36,M1G,Scarborough,Woburn
4,45,M1H,Scarborough,Cedarbrae


Create a new dataframe, pcll, that is the same as pc, and add the columns "Latitude" and "Longitude" from the coordinates dataframe. I reset the indexes a couple times, which generated new columns, so I dropped those. And now we have a dataframe showing postal code, borough, neighborhood, and coordinates!

In [243]:
pcll=pc
pcll[['Latitude','Longitude']]=coord[['Latitude','Longitude']]
pcll.drop(['index'], axis=1,inplace=True)

In [244]:
pcll

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# Exploration and Clustering

In [245]:
pcll.Borough.unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [246]:
print('The dataset has {} boroughs and {} postal codes'.format(len(pcll.Borough.unique()),pcll.Neighbourhood.count()))

The dataset has 10 boroughs and 103 postal codes


Let's use Nominatim to find the coordinates of Toronto for our map

In [247]:
from geopy.geocoders import Nominatim
import folium

In [248]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronoto are 43.6534817, -79.3839347.


Now we build the map using Folium and give it popup labels for each postal code

In [249]:
toronto_map = folium.Map(location=[latitude,longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(pcll['Latitude'], pcll['Longitude'], pcll['Borough'], pcll['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)

toronto_map

Let's narrow this down to just the boroughs that include the word 'Toronto'

In [250]:
toronto_boroughs = pcll[pcll['Borough'].str.contains('Toronto')].reset_index(drop=True)
print()
print('Number of postal codes in boroughs containing Toronto in their name:',toronto_boroughs.Borough.count())
print()
toronto_boroughs.head()


Number of postal codes in boroughs containing Toronto in their name: 39



,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Let's build the map again, this time zooming in a little more and using only the pared down data

In [251]:
toronto_map = folium.Map(location=[latitude,longitude], zoom_start=12)

for lat, lng, borough, neighbourhood in zip(toronto_boroughs['Latitude'], toronto_boroughs['Longitude'], toronto_boroughs['Borough'], toronto_boroughs['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)

toronto_map

## Segmenting neighborhoods using the Foursquare API

#### Define Foursquare credentials and version

In [252]:
CLIENT_ID = 'UCS1AMDHKOHRPWEAAZ2SD31VX0N1N23ZXL0KYIGZZEDP522A'
CLIENT_SECRET = 'ZUTLU41WL2JGX5KVNHPC0ZNX3ZYKWAFNE1DI3ECVO3SESEZP'
VERSION = '20210219' 
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UCS1AMDHKOHRPWEAAZ2SD31VX0N1N23ZXL0KYIGZZEDP522A
CLIENT_SECRET:ZUTLU41WL2JGX5KVNHPC0ZNX3ZYKWAFNE1DI3ECVO3SESEZP


#### Explore the first area in our dataframe

In [253]:
toronto_boroughs.loc[0, 'Neighbourhood']

'The Beaches'

In [254]:
neighbourhood_latitude = toronto_boroughs.loc[0,'Latitude']
neighbourhood_longitude = toronto_boroughs.loc[0,'Longitude']
neighbourhood_name = toronto_boroughs.loc[0, 'Neighbourhood']
print('Latitude and longitude for {}: {}, {}'.format(neighbourhood_name,
                                                    neighbourhood_latitude,
                                                    neighbourhood_longitude))

Latitude and longitude for The Beaches: 43.67635739999999, -79.2930312


Create the URL for the Foursquare API call:

In [255]:
limit = 100
radius = 500
url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
neighbourhood_latitude,
neighbourhood_longitude,
radius,
limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=UCS1AMDHKOHRPWEAAZ2SD31VX0N1N23ZXL0KYIGZZEDP522A&client_secret=ZUTLU41WL2JGX5KVNHPC0ZNX3ZYKWAFNE1DI3ECVO3SESEZP&v=20210219&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [256]:
results=requests.get(url).json()
#results

Now let's define a function to extract the category names from the 'categories' column. Then we'll create a dataframe of nearby venues and filter out the information we don't need. We'll also use our category function and rename the columns to something more readable.

In [257]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [258]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues=nearby_venues.loc[:,filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [259]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


I noticed later that a mail processing center was included in the dataframe as a neighborhood. I don't want that, so I dropped that row.

In [275]:
toronto_boroughs=toronto_boroughs.drop(38)
toronto_boroughs.tail()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
34,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
35,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
36,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
37,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Now let's do the same for all the neighborhoods in our search area

In [276]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
   
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
    
        # create the API request URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                            'Neighbourhood Latitude',
                            'Neighbourhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    return(nearby_venues)

In [277]:
toronto_venues=getNearbyVenues(names=toronto_boroughs['Neighbourhood'],latitudes=toronto_boroughs['Latitude'],longitudes=toronto_boroughs['Longitude'])

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [278]:
print(toronto_venues.shape)
toronto_venues.tail()

(1582, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1577,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.658906,-79.388696,Coffee Shop
1578,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Tim Hortons,43.659415,-79.391221,Coffee Shop
1579,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Understudy Café at Gerstein,43.662308,-79.394098,College Cafeteria
1580,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Hart House Music Room,43.663758,-79.395027,Music Venue
1581,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,University of Toronto Arts Centre,43.663605,-79.395159,Art Gallery


How many venues were returned for each neighbourhood?

In [279]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,60,60,60,60,60,60
Christie,16,16,16,16,16,16
Church and Wellesley,82,82,82,82,82,82
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,32,32,32,32,32,32
Davisville North,8,8,8,8,8,8


In [280]:
print('There are {} unique categories of venue.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 unique categories of venue.


## Now let's get the data ready for clustering

We'll begin with one-hot encoding the venue categories so they can be analyzed with a statistical model

In [282]:
onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')
onehot['Neighbourhood'] = toronto_venues['Neighbourhood']
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot=onehot[fixed_columns]
print('shape:',onehot.shape)
onehot.head()

shape: (1582, 236)


,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [297]:
toronto_grouped=onehot.groupby('Neighbourhood').mean().reset_index() # to the see the total number of each venue in a neighbourhood, use .sum() here; sum will display in the next step
toronto_grouped

,Neighbourhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.066667,0.066667,0.066667,0.133333,0.2,0.066667,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.016667,0.000000,0.016667,0.000000,0.016667,0.016667
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.012195,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.012195,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.024390
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.040000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.020000,0.000000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.03125,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### Let's print each neighbourhood with its 5 most common venue types

In [298]:
num_top_venues = 5 # to see the number of each venue in a neighbourhood, use .sum() instead of .mean() in the previous step
for neighbourhood in toronto_grouped['Neighbourhood']:
    print('---'+neighbourhood+'---')
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == neighbourhood].T.reset_index() # .T transposes the result so it has 2 columns instead of 237
    temp.columns = ['venue','freq'] # if viewing sum instead of mean, change 'freq' to 'sum' or 'total' or 'number'
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---Berczy Park---
          venue  freq
0   Coffee Shop  0.11
1  Cocktail Bar  0.05
2        Bakery  0.05
3   Cheese Shop  0.04
4      Beer Bar  0.04


---Brockton, Parkdale Village, Exhibition Place---
            venue  freq
0            Café  0.13
1     Coffee Shop  0.09
2  Breakfast Spot  0.09
3          Bakery  0.09
4   Grocery Store  0.04


---CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport---
                venue  freq
0     Airport Service  0.20
1      Airport Lounge  0.13
2               Plane  0.07
3  Airport Food Court  0.07
4        Airport Gate  0.07


---Central Bay Street---
                venue  freq
0         Coffee Shop  0.18
1  Italian Restaurant  0.05
2                Café  0.05
3      Sandwich Place  0.05
4        Burger Joint  0.03


---Christie---
                venue  freq
0       Grocery Store  0.25
1                Café  0.19
2                Park  0.12
3         Candy Store  0.06
4  Athletics & Spo

### Let's put that into a DF

First, we create a function to sort the values in descending order

In [307]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Then we create a new dataframe and display the top venues for each neighborhood

In [308]:
num_top_venues = 10

indicators = ['st','nd,','rd']

# create column titles for 1st, 2nd, etc. most common venue in each neighbourhood by index
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try: 
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new df
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,"2nd, Most Common Venue",3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Beer Bar,Seafood Restaurant,Cheese Shop,Restaurant,Breakfast Spot,Fish Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Bakery,Coffee Shop,Grocery Store,Furniture / Home Store,Burrito Place,Convenience Store,Stadium,Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Plane,Sculpture Garden,Coffee Shop,Boat or Ferry,Harbor / Marina,Rental Car Location,Airport Terminal,Airport Gate
3,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Bubble Tea Shop,Salad Place,Burger Joint,Thai Restaurant,Department Store,Ramen Restaurant
4,Christie,Grocery Store,Café,Park,Coffee Shop,Italian Restaurant,Baby Store,Athletics & Sports,Candy Store,Nightclub,Restaurant


## Cluster the Neighbourhoods

Let's use k-means to cluster the neighbourhoods into 5 clusters

In [316]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

kmeans.labels_[0:30]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 2, 4, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the top 10 venues as well as the cluster label for each neighbourhood

In [323]:
#neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_boroughs

toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.drop('Postal Code', axis=1, inplace=True)

toronto_merged.head()

,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,"2nd, Most Common Venue",3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,The Beaches,43.676357,-79.293031,3,Health Food Store,Trail,Pub,Neighborhood,Convenience Store,Cosmetics Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Fast Food Restaurant,Pet Store,Ice Cream Shop,Pizza Place,Coffee Shop,Movie Theater,Pub,Restaurant,Sandwich Place
3,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Café,American Restaurant,Bakery,Gastropub,Brewery,Coworking Space,Latin American Restaurant,Bookstore,Cheese Shop
4,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Business Service,Swim School,Yoga Studio,Distribution Center,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


Now let's view the clusters on a map

In [330]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme
x = np.arange(kclusters)
ys = [i + x +(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon],
                       radius=5,
                       popup=label,
                       color=rainbow[cluster-1],
                       fill=True,
                       fill_color=rainbow[cluster-1],
                       fill_opacity=0.7).add_to(map_clusters)

map_clusters

## Now we'll examine the clusters

### Cluster 1

Coffeeshops, Cafes, and Restaurants cluste

In [347]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,"2nd, Most Common Venue",3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Restaurant,Cosmetics Shop,Brewery,Bubble Tea Shop,Café
2,"India Bazaar, The Beaches West",0,Park,Fast Food Restaurant,Pet Store,Ice Cream Shop,Pizza Place,Coffee Shop,Movie Theater,Pub,Restaurant,Sandwich Place
3,Studio District,0,Coffee Shop,Café,American Restaurant,Bakery,Gastropub,Brewery,Coworking Space,Latin American Restaurant,Bookstore,Cheese Shop
4,Lawrence Park,0,Park,Bus Line,Business Service,Swim School,Yoga Studio,Distribution Center,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store
5,Davisville North,0,Pizza Place,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Breakfast Spot,Park,Gym / Fitness Center,Distribution Center,Ethiopian Restaurant
6,"North Toronto West, Lawrence Park",0,Coffee Shop,Clothing Store,Sporting Goods Shop,Yoga Studio,Bagel Shop,Ice Cream Shop,Fast Food Restaurant,Mexican Restaurant,Diner,Park
7,Davisville,0,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Gym,Café,Italian Restaurant,Sushi Restaurant,Greek Restaurant,Farmers Market
9,"Summerhill West, Rathnelly, South Hill, Forest...",0,Coffee Shop,Pizza Place,Fried Chicken Joint,Pub,Restaurant,Liquor Store,Bank,Supermarket,Sushi Restaurant,Bagel Shop
11,"St. James Town, Cabbagetown",0,Restaurant,Bakery,Coffee Shop,Café,Pub,Italian Restaurant,Chinese Restaurant,Pizza Place,Park,Outdoor Sculpture
12,Church and Wellesley,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Pub,Men's Store,Mediterranean Restaurant,Hotel


### Cluster 2

Activities cluster

In [348]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,"2nd, Most Common Venue",3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park, Summerhill East",1,Playground,Summer Camp,Tennis Court,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant


### Cluster 3

Playgrounds cluster

In [345]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,"2nd, Most Common Venue",3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Rosedale,Park,Playground,Trail,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant
23,"Forest Hill North & West, Forest Hill Road Park",Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Discount Store,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


### Cluster 4

Crunchy cluster

In [349]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,"2nd, Most Common Venue",3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,3,Health Food Store,Trail,Pub,Neighborhood,Convenience Store,Cosmetics Shop,Event Space,Ethiopian Restaurant,Escape Room,Electronics Store


### Cluster 5

Gardens and Ice cream cluster

In [352]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,"2nd, Most Common Venue",3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,4,Garden,Home Service,Ice Cream Shop,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
